# Create Knowledge Bases 

In this notebook, we will learn how to configure and use a Knowledge Base (KB) in AWS Bedrock, with the aim of implementing a Retrieval-Augmented Generation (RAG) architecture.

The RAG approach combines the power of generative models with external information sources, allowing the model to respond with up-to-date, accurate, and data-driven context.

For this example, you must have the following template enabled in your AWS account:

- amazon.titan-embed-text-v2:0


This model will be used to generate vector representations of the text and enable efficient semantic search within the Knowledge Base.

Notebook contents

- Initial setup and connection to Bedrock.

- Creation and configuration of the Knowledge Base.

- Document ingestion.
- Consulta mediante RAG.

In [19]:
#!pip install requests-aws4auth
#!pip install opensearch-py

In [5]:
import os
import sys
import json
import time
import random

import boto3
from requests_aws4auth import AWS4Auth
from opensearchpy import OpenSearch, RequestsHttpConnection

In [37]:
# Create boto3 session 
boto_session = boto3.Session()
aws_region = boto_session.region_name

aoss_client = boto3.client('opensearchserverless')
bedrock_agent_client = boto3.client('bedrock-agent')
s3_client = boto3.client('s3')

In [29]:
# Set the Bedrock models 
model_id = 'eu.anthropic.claude-3-7-sonnet-20250219-v1:0'
model_arn = f'arn:aws:bedrock:{aws_region}::foundation-model/{model_id}'

embedding_model_id = 'amazon.titan-embed-text-v2:0'
embedding_model_arn = f'arn:aws:bedrock:{aws_region}::foundation-model/{embedding_model_id}'
embedding_model_dim = 1024

In [ ]:
# Define names 
aoss_collection_name = f"bedrock-kb-collection-workshop"
aoss_index_name = "bedrock-kb-index-workshop"
bedrock_kb_name = "bedrock-kb-workshop"

# Choose data

In [9]:
s3_bucket_name = 'ia-aws-oct-2025' 
path = "rag"

## Create KB

First, we need to create the vectors in the database in Amazon OpenSearch Serverless (AOSS). To do this, we must grant permissions to the invocation role.

In [ ]:
bedrock_kb_execution_role_arn = "arn:aws:iam::XXXXXXXX:role/AmazonBedrockExecutionRoleForKnowledgeBase_XXXX"

In [ ]:
# Request to create AOSS collection
aoss_collection = aoss_client.create_collection(name=aoss_collection_name, type='VECTORSEARCH')

# Wait until collection becomes active
time.sleep(5)

## Vector Index

Define vector index into AOSS

In [20]:
# Use default credential configuration for authentication
credentials = boto_session.get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    aws_region,
    'aoss',
    session_token=credentials.token)

# Construct AOSS endpoint host
host = f"{aoss_collection['createCollectionDetail']['id']}.{aws_region}.aoss.amazonaws.com"

# Build the OpenSearch client
os_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

In [ ]:
# Define the configuration for the AOSS vector index
index_definition = {
   "settings": {
      "index.knn": "true",
       "number_of_shards": 1,
       "knn.algo_param.ef_search": 512,
       "number_of_replicas": 0,
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": embedding_model_dim,
             "method": {
                 "name": "hnsw",
                 "engine": "faiss",
                 "space_type": "l2"
             },
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"
         }
      }
   }
}

# Create an OpenSearch index
response = os_client.indices.create(index=aoss_index_name, body=index_definition)

# Waiting for index creation to propagate
time.sleep(40)

print("A new AOSS index created:", json.dumps(response, indent=2))

# Knowledge Base

Once we have the vectors, we can define our database based on them.

In [ ]:
# Vector Storage Configuration
storage_config = {
    "type": "OPENSEARCH_SERVERLESS",
    "opensearchServerlessConfiguration": {
        "collectionArn": aoss_collection["createCollectionDetail"]['arn'],
        "vectorIndexName": aoss_index_name,
        "fieldMapping": {
            "vectorField": "vector",
            "textField": "text",
            "metadataField": "text-metadata"
        }
    }
}

# Knowledge Base Configuration
knowledge_base_config = {
    "type": "VECTOR",
    "vectorKnowledgeBaseConfiguration": {
        "embeddingModelArn": embedding_model_arn
    }
}

response = bedrock_agent_client.create_knowledge_base(
    name=bedrock_kb_name,
    description="Amazon shareholder letter knowledge base.",
    roleArn=bedrock_kb_execution_role_arn,
    knowledgeBaseConfiguration=knowledge_base_config,
    storageConfiguration=storage_config)

bedrock_kb_id = response['knowledgeBase']['knowledgeBaseId']

print("Waiting until BKB becomes active: ", end='')
time.sleep(120)

In [ ]:
response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=bedrock_kb_id)

print(json.dumps(response['knowledgeBase'], indent=2, default=str))

# Connect KB with database

In [ ]:
# KB and Vector Ingestion Configuration
response = bedrock_agent_client.create_data_source(
    name=bedrock_kb_name,
    description="Amazon shareholder letter knowledge base.",
    knowledgeBaseId=bedrock_kb_id,
    dataSourceConfiguration={
        "type": "S3",
        "s3Configuration":{
            "bucketArn": f"arn:aws:s3:::{s3_bucket_name}",
        }
    },
    vectorIngestionConfiguration={
        "chunkingConfiguration": {
            "chunkingStrategy": "FIXED_SIZE",
            "fixedSizeChunkingConfiguration": {
                "maxTokens": 512,
                "overlapPercentage": 20
            }
        }
    }
)

bedrock_ds_id = response['dataSource']['dataSourceId']

bedrock_ds_id

In [ ]:
# Start an ingestion job
response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId=bedrock_kb_id, dataSourceId=bedrock_ds_id)

bedrock_job_id = response['ingestionJob']['ingestionJobId']
bedrock_job_id

# Wait until ingestion job completes
time.sleep(120)

# Retrive and generate

Once our KB is available, we can use *retrieve_and_generate* to perform RAG on it.

In [ ]:
bedrock_agent_client = boto3.client('bedrock-agent-runtime')

In [24]:
user_query = "What is FP32?"

In [ ]:
# retrieve and generate
response = bedrock_agent_client.retrieve_and_generate(
    input={
        'text': user_query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': bedrock_kb_id,
            'modelArn': model_arn
        }
    }
)

In [ ]:
print("Final reply:\n", response['output']['text'])

# Clean 

In [ ]:
# Delete AOSS 

# aoss_client.delete_collection(id=aoss_collection['createCollectionDetail']['id'])
# aoss_client.delete_access_policy(type="data", name=aoss_access_policy['accessPolicyDetail']['name'])
# aoss_client.delete_security_policy(type="network", name=aoss_network_policy['securityPolicyDetail']['name'])
# aoss_client.delete_security_policy(type="encryption", name=aoss_encryption_policy['securityPolicyDetail']['name'])